# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/1f5yE/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298590244290                   -0.85    5.2         
  2   -8.300218659346       -2.79       -1.25    1.0    100ms
  3   -8.300437636300       -3.66       -1.89    2.1    118ms
  4   -8.300461824565       -4.62       -2.77    3.1    136ms
  5   -8.300464483310       -5.58       -3.19    3.0    152ms
  6   -8.300464574321       -7.04       -3.36    4.1    165ms
  7   -8.300464608472       -7.47       -3.50    9.1    294ms
  8   -8.300464628143       -7.71       -3.65    2.4    142ms
  9   -8.300464638530       -7.98       -3.81    1.1    110ms
 10   -8.300464642148       -8.44       -3.97    1.2    114ms
 11   -8.300464643769       -8.79       -4.27    1.6    125ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67470064185                   -0.70    6.4         
  2   -16.67876328400       -2.39       -1.14    1.2    272ms
  3   -16.67921557277       -3.34       -1.87    1.5    290ms
  4   -16.67928087111       -4.19       -2.77    4.9    367ms
  5   -16.67928587281       -5.30       -3.18    3.2    388ms
  6   -16.67928620187       -6.48       -3.48    3.4    338ms
  7   -16.67928621949       -7.75       -3.98    2.0    265ms
  8   -16.67928622147       -8.70       -4.59    3.4    327ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32398345823                   -0.56    7.1         
  2   -33.33256182418       -2.07       -1.00    1.0    973ms
  3   -33.33407385522       -2.82       -1.74    3.1    1.19s
  4   -33.33521647714       -2.94       -2.28    3.4    1.25s
  5   -33.33690871286       -2.77       -2.51   13.8    1.85s
  6   -33.33693312685       -4.61       -2.83    4.0    1.20s
  7   -33.33694366406       -4.98       -3.74    2.6    1.17s
  8   -33.33694370962       -7.34       -3.90    5.0    1.54s
  9   -33.33694377502       -7.18       -4.39    2.0    982ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298626624649                   -0.85    5.0         
  2   -8.300270708138       -2.78       -1.59    1.2   86.4ms
  3   -8.300426234289       -3.81       -2.58    2.8    113ms
  4   -8.300388702430   +   -4.43       -2.31    4.8    170ms
  5   -8.300464299504       -4.12       -3.37    1.1    115ms
  6   -8.300464567273       -6.57       -3.80    3.9    157ms
  7   -8.300464640327       -7.14       -4.24    2.1    118ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32389597574                   -0.56    7.2         
  2   -33.30585685539   +   -1.74       -1.26    1.0    838ms
  3   -7.215992927599   +    1.42       -0.36    6.2    1.82s
  4   -33.33320792819        1.42       -1.89    6.4    1.72s
  5   -33.12581642657   +   -0.68       -1.26    4.5    1.63s
  6   -32.50218933012   +   -0.21       -1.10    4.1    1.45s
  7   -33.32449101669       -0.08       -1.96    4.4    1.39s
  8   -33.33662074779       -1.92       -2.33    2.8    1.05s
  9   -33.33649187548   +   -3.89       -2.64    2.8    1.08s
 10   -33.33667556353       -3.74       -2.56    2.6    1.22s
 11   -33.33692722763       -3.60       -3.27    2.0    1.01s
 12   -33.33693195222       -5.33      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.